### UniprotファイルからFT行にTRANSMEMがあるエントリを抽出する

In [ ]:
import sys

def extract_transmembrane_entries(input_file, output_file):
    total_entries = 0
    transmem_entries = 0
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        entry_buffer = []
        is_transmem = False
        for line in f_in:
            entry_buffer.append(line)
            if line.startswith('FT') and 'TRANSMEM' in line:
                is_transmem = True
            
            if line.strip() == '//':
                total_entries += 1
                if is_transmem:
                    transmem_entries += 1
                    f_out.write("".join(entry_buffer))
                entry_buffer = []
                is_transmem = False
    return total_entries, transmem_entries

if __name__ == '__main__':
    input_filename = 'uniprot_sprot_human.dat'
    output_filename = 'uniprot_sprot_human_transmem.dat'
    total, transmem = extract_transmembrane_entries(input_filename, output_filename)
    print(f"全エントリ数: {total}")
    print(f"抽出された膜貫通タンパク質のエントリ数: {transmem}")
    print(f"抽出したエントリは {output_filename} に保存されました。") 

### UniprotファイルからACを1つずつ抽出する

In [ ]:
import sys

def extract_primary_accession_numbers(input_file):
    """
    Uniprotデータファイルから各エントリの最初の(プライマリ)アクセッション番号(AC)を抽出する。
    """
    accession_numbers = []
    with open(input_file, 'r') as f:
        ac_found_for_this_entry = False
        for line in f:
            # '//'はエントリの区切り
            if line.strip() == '//':
                ac_found_for_this_entry = False
                continue

            # AC行で、かつこのエントリでまだACを見つけていない場合
            if line.startswith('AC') and not ac_found_for_this_entry:
                try:
                    # AC   P04637; Q16254; -> P04637
                    ac = line.split()[1].rstrip(';')
                    accession_numbers.append(ac)
                    ac_found_for_this_entry = True
                except IndexError:
                    # AC行が空など、予期せぬフォーマットの場合はスキップ
                    pass
    return accession_numbers

if __name__ == '__main__':
    input_filename = 'uniprot_sprot_human_transmem.dat'
    output_filename = 'accession_transmem.txt'
    try:
        accessions = extract_primary_accession_numbers(input_filename)
        
        with open(output_filename, 'w') as f_out:
            for ac in accessions:
                f_out.write(ac + '\n')

        print(f"ファイル '{input_filename}' から {len(accessions)} 個のアクセッション番号を抽出し、'{output_filename}' に保存しました。")

    except FileNotFoundError:
        print(f"エラー: 入力ファイル '{input_filename}' が見つかりません。")
        sys.exit(1) 

### Gタンパク質共役型受容体を取り出す

In [ ]:
import sys

def extract_gpcr_accessions(input_file):
    """
    Uniprotデータファイルから、'G protein-coupled receptor' に関連する
    エントリのプライマリACを抽出する。

    条件:
    - DR行に 'GO;' が含まれる
    - DR行に 'G protein-coupled receptor' が含まれる
    """
    gpcr_acs = []
    
    with open(input_file, 'r') as f:
        current_entry_lines = []
        for line in f:
            current_entry_lines.append(line)
            
            # '//' はエントリの終わりを示す
            if line.strip() == '//':
                # エントリが条件を満たすかチェック
                is_gpcr = False
                primary_ac = None
                
                for entry_line in current_entry_lines:
                    # エントリの最初のAC番号を取得
                    if entry_line.startswith('AC') and primary_ac is None:
                        try:
                            primary_ac = entry_line.split()[1].rstrip(';')
                        except IndexError:
                            pass # 不正なAC行をスキップ
                    
                    # GPCRの条件をチェック
                    if (entry_line.startswith('DR') and 
                        'GO;' in entry_line and 
                        'G protein-coupled receptor' in entry_line):
                        is_gpcr = True

                # 条件を満たしていれば、ACをリストに追加
                if is_gpcr and primary_ac:
                    gpcr_acs.append(primary_ac)
                
                # 次のエントリのためにバッファをリセット
                current_entry_lines = []
                
    return gpcr_acs

if __name__ == '__main__':
    # 全てのタンパク質が含まれるファイルを入力とする
    input_filename = 'uniprot_sprot_human.dat'
    output_filename = 'ac_gpc_receptors.txt'
    
    try:
        accessions = extract_gpcr_accessions(input_filename)
        
        with open(output_filename, 'w') as f_out:
            for ac in accessions:
                f_out.write(ac + '\n')
                
        print(f"'{input_filename}' から {len(accessions)} 個の G protein-coupled receptor のACを抽出し、")
        print(f"'{output_filename}' に保存しました。")
        
    except FileNotFoundError:
        print(f"エラー: 入力ファイル '{input_filename}' が見つかりません。")
        sys.exit(1) 